<a href="https://colab.research.google.com/github/towardsai/ai-tutor-rag-system/blob/main/notebooks/17-Using_LLMs_to_rank_chunks_as_the_Judge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Packages and Setup Variables


In [ ]:
!pip install -q huggingface-hub==0.34.4 llama-index==0.13.3 llama-index-llms-openai==0.5.4 llama-index-vector-stores-chroma==0.5.2 \
                llama-index-llms-google-genai==0.3.0 chromadb==1.0.20 jedi==0.19.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.4 MB/s eta 0:00:0

In [ ]:
import os

# Set the following API Keys in the Python environment. Will be used later.
# os.environ["OPENAI_API_KEY"] = "[OPENAI_API_KEY]"
# os.environ["GOOGLE_API_KEY"] = "<YOUR_API_KEY>"

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["GOOGLE_API_KEY"] = userdata.get('Google_api_key')

# Load a Model


In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-5-mini", additional_kwargs={'reasoning_effort':'minimal'})
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

# Load Indexes


Downloading vector store from Huggingface hub

In [ ]:
from huggingface_hub import hf_hub_download
vectorstore = hf_hub_download(repo_id="jaiganesan/ai_tutor_knowledge", filename="vectorstore.zip",repo_type="dataset",local_dir="/content")

vectorstore.zip:   0%|          | 0.00/97.2M [00:00<?, ?B/s]

In [ ]:
!unzip /content/vectorstore.zip

Archive:  /content/vectorstore.zip
   creating: ai_tutor_knowledge/
   creating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/length.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/index_metadata.pickle  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/link_lists.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/header.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/data_level0.bin  
  inflating: ai_tutor_knowledge/chroma.sqlite3  


In [ ]:
# Load the vector store from the local storage.
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

db2 = chromadb.PersistentClient(path="/content/ai_tutor_knowledge")
chroma_collection = db2.get_or_create_collection("ai_tutor_knowledge")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding

# Create the index based on the vector store.
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

In [ ]:
query_engine = index.as_query_engine(similarity_top_k=10)

res = query_engine.query("Explain how Advance RAG works?")

In [ ]:
res.response

'The material doesn’t mention anything called “Advance RAG.” Based on the provided information, there are several closely related RAG variants described (Adaptive RAG, Corrective RAG, Self‑RAG / Self‑Reflective RAG, Speculative RAG). If you meant one of those, here are concise explanations so you can pick the intended one:\n\n- Adaptive RAG: Routes a question to different retrieval strategies depending on the question type or need (i.e., select the most appropriate retriever or indexing approach), improving the chance of retrieving relevant documents.\n\n- Corrective RAG (CRAG): Acts as a fallback / retrieval‑quality filter. An external evaluator checks retrieved documents and, when they are low‑relevance, triggers fallback retrieval (for example, a web search) to get better evidence before generation.\n\n- Self‑Reflective RAG (Self‑RAG): Instruction‑tunes a model to generate self‑reflection tags that guide dynamic retrieval and let the model critique document relevance during generati

# RankGPT


In [ ]:
from llama_index.core.postprocessor.rankGPT_rerank import RankGPTRerank

rankGPT = RankGPTRerank(top_n=3,llm=Settings.llm, verbose=True)

In [ ]:
# Define a query engine that is responsible for retrieving related pieces of text,
# and using a LLM to formulate the final answer.
# The `node_postprocessors` function will be applied to the retrieved nodes.
query_engine = index.as_query_engine(similarity_top_k=10, node_postprocessors=[rankGPT])

res = query_engine.query("Explain how Retrieval Augmented Generation (RAG) works?")

After Reranking, new rank list for nodes: [1, 3, 0, 4, 6, 8, 9, 7, 2, 5]

In [ ]:
res.response

'Retrieval-Augmented Generation (RAG) is a two-part approach that combines external information retrieval with language generation to produce more accurate, up-to-date, and grounded responses. Here’s how it works, step by step:\n\n1. Two main components\n- Retrieval: fetches relevant documents or passages from external knowledge sources (non‑parametric memory).\n- Generation: uses a pretrained large language model (parametric memory) to produce the final response conditioned on the query and retrieved content.\n\n2. Retrieval component details\n- Indexing: source documents are organized for efficient lookup. This can use sparse inverted indexes or dense vector encodings (embeddings) stored in a vector index.\n- Searching: the user query is used to search the index to obtain candidate documents or chunks. Neural retrievers (dense) or traditional retrieval (sparse) can be used.\n- Optional reranking: retrieved candidates can be reranked by a specialized model to improve the ordering of m

In [ ]:
# Show the retrieved nodes
for src in res.source_nodes:
    print("Node ID\t", src.node_id)
    print("Title\t", src.metadata["title"])
    print("Text\t", src.text.strip())
    print("Score\t", src.score)
    print("-_" * 20)

Node ID	 1c324686-fad7-4a41-bfd3-d44f9612ca91
Title	 Evaluation of Retrieval-Augmented Generation: A Survey:Research Paper Information
Text	 Authors: Hao Yu, Aoran Gan, Kai Zhang, Shiwei Tong, Qi Liu, and Zhaofeng Liu.Numerous studies of Retrieval-Augmented Generation (RAG) systems have emerged from various perspectives since the advent of Large Language Models (LLMs). The RAG system comprises two primary components: Retrieval and Generation. The retrieval component aims to extract relevant information from various external knowledge sources. It involves two main phases, indexing and searching. Indexing organizes documents to facilitate efficient retrieval, using either inverted indexes for sparse retrieval or dense vector encoding for dense retrieval. The searching component utilizes these indexes to fetch relevant documents based on the user's query, often incorporating the optional rerankers to refine the ranking of the retrieved documents. The generation component utilizes the retr

# Custom Postprocessor


## The `Judger` Function


The following function will query GPT-5 (mini) to retrieve the top three nodes that has highest similarity to the asked question.


In [ ]:
from pydantic import BaseModel
from llama_index.llms.openai import OpenAI
from llama_index.core.prompts import PromptTemplate


def judger(nodes, query):

    # The model's output template
    class OrderedNodes(BaseModel):
        """A node with the id and assigned score."""

        node_id: list
        score: list

    # Prepare the nodes and wrap them in <NODE></NODE> identifier, as well as the query
    the_nodes = ""
    for idx, item in enumerate(nodes):
        the_nodes += f"<NODE{idx+1}>\nNode ID: {item.node_id}\nText: {item.text}\n</NODE{idx+1}>\n"

    query = "<QUERY>\n{}\n</QUERY>".format(query)

    # Define the prompt template
    prompt_tmpl = PromptTemplate(
        """
    You receive a qurey along with a list of nodes' text and their ids. Your task is to assign score
    to each node based on its contextually closeness to the given query. The final output is each
    node id along with its proximity score.
    Here is the list of nodes:
    {nodes_list}

    And the following is the query:
    {user_query}

    Score each of the nodes based on their text and their relevancy to the provided query.
    The score must be a decimal number between 0 an 1 so we can rank them."""
    )

    # Define the an instance of GPT-5 mini and send the request
    llm = OpenAI(model="gpt-5-mini", additional_kwargs={'reasoning_effort':'minimal'})
    ordered_nodes = llm.structured_predict(
        OrderedNodes, prompt_tmpl, nodes_list=the_nodes, user_query=query
    )

    return ordered_nodes

## Define Postprocessor


The following class will use the `judger` function to rank the nodes, and filter them based on the ranks.


In [ ]:
from typing import List, Optional
from llama_index.core import QueryBundle
from llama_index.core.postprocessor.types import BaseNodePostprocessor
from llama_index.core.schema import NodeWithScore


class OpenaiAsJudgePostprocessor(BaseNodePostprocessor):
    def _postprocess_nodes(
        self, nodes: List[NodeWithScore], query_bundle: Optional[QueryBundle]
    ) -> List[NodeWithScore]:

        r = judger(nodes, query_bundle)

        node_ids = r.node_id
        scores = r.score

        sorted_scores = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)
        num_nodes_to_select = min(3, len(sorted_scores))
        top_nodes = [sorted_scores[i][0] for i in range(num_nodes_to_select)]
        # top_nodes = [sorted_scores[i][0] for i in range(3)]

        selected_nodes_id = [node_ids[item] for item in top_nodes]

        final_nodes = []
        for item in nodes:
            if item.node_id in selected_nodes_id:
                final_nodes.append(item)

        return final_nodes

In [ ]:
judge = OpenaiAsJudgePostprocessor()

## Query Engine with Postprocessor


In [ ]:
# Define a query engine that is responsible for retrieving related pieces of text,
# and using a LLM to formulate the final answer.
# The `node_postprocessors` function will be applied to the retrieved nodes.
query_engine = index.as_query_engine(similarity_top_k=10, node_postprocessors=[judge])

res = query_engine.query("Compare Retrieval Augmented Generation (RAG) and Parameter efficient Finetuning (PEFT)")

In [ ]:
res.response

'Retrieval-Augmented Generation (RAG) and Parameter-Efficient Fine-Tuning (PEFT) are different techniques addressing complementary problems in working with large language models. Comparing them along key dimensions:\n\nPurpose\n- RAG: Augments generation by giving the model access to external, non-parametric knowledge (e.g., a dense vector index of Wikipedia). It improves factuality, specificity, and diversity for knowledge‑intensive tasks by retrieving relevant documents and conditioning generation on them.\n- PEFT: Focuses on updating a pre-trained model for a downstream task while changing only a small subset of model parameters (or adding small adapters), to make fine-tuning cheaper and more parameter-efficient.\n\nHow they work\n- RAG: Combines a neural retriever (dense vector index) with a seq2seq generator. Retrieved passages are passed to the seq2seq model and the output is marginalized over retrieved documents. Two formulations exist: one conditions on the same retrieved passa

In [ ]:
# Show the retrieved nodes
for src in res.source_nodes:
    print("Node ID\t", src.node_id)
    print("Title\t", src.metadata["title"])
    print("Text\t", src.text)
    print("Score\t", src.score)
    print("-_" * 20)

Node ID	 40f472bb-265c-438a-bb45-25be7c023dc8
Title	 RAG
Text	 extractive downstream tasks. We explore ageneral-purpose fine-tuning recipe for retrieval-augmented generation (RAG) — models which combine pre-trainedparametric and non-parametric memory for language generation. We introduce RAG models where the parametric memory is apre-trained seq2seq model and the non-parametric memory is a dense vector index of Wikipedia, accessed with apre-trained neural retriever. We compare two RAG formulations, one which conditions on the same retrieved passagesacross the whole generated sequence, the other can use different passages per token. We fine-tune and evaluate ourmodels on a wide range of knowledge-intensive NLP tasks and set the state-of-the-art on three open domain QA tasks,outperforming parametric seq2seq models and task-specific retrieve-and-extract architectures. For language generationtasks, we find that RAG models generate more specific, diverse and factual language than a state-of